In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("joint1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("joint2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2

In [6]:
x = MechanismState(Float64, mechanism);

In [7]:
rand!(x)
setdirty!(x)
x.q[joint1]

7-element Array{Float64,1}:
 0.713825
 0.196429
 0.449078
 0.500198
 0.987024
 0.745708
 0.964063

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
relative_transform(x, joint2.frameAfter, body1.frame)

Transform3D from "after_joint2" to "body1":
rotation: 2.4910179488517166 rad about [0.07030232384199758,-0.32292825284800697,0.9438087342120481], translation: Vec(0.3096386810558378,0.9555377460234,0.5790172111932055)


In [11]:
center_of_mass(x)

Point3D in "world": Vec(1.7857715019360747,2.983706266514492,1.4200300906645418)

In [12]:
mass(mechanism)

0.2789018422850955

In [13]:
transform(x, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(0.9453600986104389,2.604266191589542,1.1308069408306305)

In [14]:
relative_twist(x, body2, body1)

Twist of "after_joint2" w.r.t "after_joint1" in "world":
angular: Vec(0.46784312907743253,-0.1948719271727295,-0.09643538925954553), linear: Vec(-0.03078089610144799,0.6202064266806213,-1.4026121883122376)

In [15]:
relative_twist(x, body1, world) + relative_twist(x, body2, body1) + relative_twist(x, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [16]:
p = path(mechanism, body2, world)
J = geometric_jacobian(x, p)

GeometricJacobian: body: "world", base: "after_joint2", expressed in "world":
6x7 Array{Float64,2}:
 -0.60461     0.454519  -0.870539  -0.188612  -0.0       -0.0       -0.0     
  0.25184    -0.542411  -0.10254   -0.833832  -0.0       -0.0       -0.0     
  0.124627   -0.706543  -0.481298   0.518796  -0.0       -0.0       -0.0     
  0.0397792  -1.48023   -1.09166    1.47147    0.454519  -0.870539  -0.188612
 -0.801514    1.22418    0.465544  -0.85358   -0.542411  -0.10254   -0.833832
  1.81264    -1.89203    1.87533   -0.836947  -0.706543  -0.481298   0.518796

In [17]:
Twist(J, velocity_vector(x, p))

Twist of "world" w.r.t "after_joint2" in "world":
angular: Vec(-0.9130860882045293,-0.3024885894618244,-0.05341214693575047), linear: Vec(-0.03381610759390795,-1.4827377098829775,1.2242204694869692)

In [18]:
mass_matrix(x)

7x7 Array{Float64,2}:
  1.39026      -1.31766     -1.35752      …   0.228936     -0.351461  
 -1.31766       3.55713      1.3327          -0.284316      1.06016   
 -1.35752       1.3327       2.00143          0.0           0.675253  
 -2.77556e-17   0.00399594  -0.228936         9.71445e-17   0.00504227
 -0.00399594    0.0          0.284316         5.55112e-17   0.00361359
  0.228936     -0.284316     3.64292e-17  …   0.278902     -0.0296021 
 -0.351461      1.06016      0.675253        -0.0296021     0.611205  

In [19]:
A = momentum_matrix(x)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
 -2.37625     4.37116     2.12618    …   0.359907   -0.76196     1.26499   
 -0.315694    0.513705    1.21708        0.105063    0.333089    0.447998  
  1.25147    -0.140743   -2.32239       -0.673358    0.258338   -0.110398  
  0.0397015  -0.0554417   0.351563       0.242795    0.0526044  -0.00472936
  0.190484   -0.234904   -0.0950235      0.0285985   0.232557   -0.0215776 
 -0.120694    0.150325   -0.0249124  …   0.134235   -0.144693    0.0206592 

In [20]:
Momentum(A, velocity_vector(x))

Momentum expressed in "world":
angular: Vec(3.6965591868277894,0.7880453164035877,-0.3351988107268776), linear: Vec(0.08663851482558117,0.12215919745483934,0.1885407817701067)

In [21]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(x, accel, world.frame)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(1.2235710262077952,-0.5901644916398131,0.4950344026100619), linear: Vec(2.329911350401585,0.14103487124968894,-3.4949172834044266)

In [22]:
bias_acceleration(x, body2)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(-0.018762076863519238,0.11304231829664455,-0.3194520538870642), linear: Vec(-0.8389553295924949,0.7095795369222647,-0.12466572998784797)

In [23]:
v̇ = [j::Joint => rand(num_velocities(j))::Vector{Float64} for j in joints(mechanism)]
τ = inverse_dynamics(x, v̇)

7-element Array{Float64,1}:
 -1.56747
  3.65742
  1.13159
  2.09567
  1.63368
 -1.3787 
  1.02628

In [27]:
relative_acceleration(x, body1, body2, v̇)

SpatialAcceleration of "after_joint1" w.r.t "after_joint2" in "world":
angular: Vec(-0.12333262784620072,-0.053855237784364476,0.3487416967368453), linear: Vec(0.8483041947791244,-0.8979504749160285,0.5506712697368239)

In [24]:
rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [25]:
rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body3, Edge: joint3
      Vertex: body4, Edge: joint4
        Vertex: body7, Edge: joint7
      Vertex: body6, Edge: joint6
        Vertex: body8, Edge: joint8
          Vertex: body9, Edge: joint9
        Vertex: body10, Edge: joint10
  Vertex: body2, Edge: joint2
    Vertex: body5, Edge: joint5
      Vertex: body11, Edge: joint11